# BertSum

## Import library

In [1]:
# # colab
# from google.colab import drive
# drive.mount('/content/drive')
# PROJECT_ID = "/Projects/202011_DACON_Summarization"

In [18]:
# !pwd
!cd ,

D:\ssum-workspace\004_mois\txt_summ_prj\working\_git_src\KoBertSum-master\tutorials


In [5]:
# ROOT_DIR = "/home/uoneway/Project"
ROOT_DIR = r"D:\ssum-workspace\004_mois\txt_summ_prj\working\_git_src\KoBertSum-master"

In [19]:
import numpy as np 
import pandas as pd
import os
import json

# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
# sns.set(style='white', context='notebook', palette='deep')

import warnings
warnings.filterwarnings(action='ignore')

if get_ipython():
    print("jupyter envirionment")
    from tqdm import tqdm_notebook as tqdm  #이거 안해주면 한 줄씩 출력됨 ;;
else:
    print("command shell envirionment")
    from tqdm import tqdm

jupyter envirionment


In [20]:
PROJECT_DIR = ROOT_DIR + '/ext'
DATA_DIR = PROJECT_DIR + '/data'
RAW_DATA_DIR = DATA_DIR + '/raw'
JSON_DATA_DIR = DATA_DIR + '/json_data'
BERT_DATA_DIR = DATA_DIR + '/bert_data' 

MODEL_DIR = PROJECT_DIR + '/models'
LOG_DIR = PROJECT_DIR + '/logs' # logs -> for storing logs information during preprocess and finetuning
RESULT_DIR = PROJECT_DIR + '/result'


## preprocessing

Our data process includes the following four steps:  
- 1. Convert our raw data to **.story** format and store in */kaggle/working/news_sum/* . This enables us to use **preprocess.py** code to do further process  
- 2. Tokenize **.story** file via using [**StanfordCoreNLP**](https://stanfordnlp.github.io/CoreNLP/index.html). Stanford CoreNLP is written in **Java**; recent releases require Java **1.8+**. You need to have Java installed to run CoreNLP. However, you can interact with CoreNLP via the command-line or its web service; many people use CoreNLP while writing their own code in Javascript, Python, or some other language.  

    (*Here we activate Stanford CoreNLP by the code in **preprocess.py***)
- 3. Convert tokenized data as **json** format 
- 4. Convert **json** file to **.pt** format for model finetuning


**folder name -> data file**
https://www.kaggle.com/uoneway/text-summarization-with-presumm-finetuning/edit




### Import data

In [23]:
with open(RAW_DATA_DIR + '/train.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

trains = []
for json_str in json_list:
    line = json.loads(json_str)
    trains.append(line)

train_df = pd.DataFrame(trains)
train_df['extractive_sents'] = train_df.apply(lambda row: list(np.array(row['article_original'])[row['extractive']]) , axis=1)
train_df

,media,id,article_original,abstractive,extractive,extractive_sents
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]",[24일 통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 ...
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]",[가충순 의원과 이수의 의원은 16일 팔봉면 폰타나 리조트에서 열린 한국지역신문협회...
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]",[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]",[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]",[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...
...,...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]",[두통은 크게 ▲근육 수축에 의한 긴장성 두통 ▲혈관의 수축과 이완에 따른 혈관성 ...
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]",[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]",[양산시는 “지난해 10월부터 43억 원을 들여 한일아파트 뒤편 옹벽에 대한 보수·...
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]","[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일..."


In [24]:
train_df[train_df['article_original'] ==

SyntaxError: unexpected EOF while parsing (<ipython-input-24-3e1fbc465259>, line 1)

### DIR 만들기

In [27]:
import os 
os.chdir(DATA_DIR)

!cd ,

!mkdir json_data json_data\train_ext json_data\train_abs json_data\test
!mkdir bert_data bert_data\train_ext bert_data\train_abs bert_data\test
# !mkdir logs
# !mkdir temp
# !mkdir bertsumextabs 

D:\ssum-workspace\004_mois\txt_summ_prj\working\_git_src\KoBertSum-master\ext\data


하위 디렉터리 또는 파일 json_data이(가) 이미 있습니다.
다음 내용 진행 중 오류 발생: json_data.
하위 디렉터리 또는 파일 bert_data이(가) 이미 있습니다.
다음 내용 진행 중 오류 발생: bert_data.


In [ ]:
# !mv bert_data/train.* bert_data/train_ext

In [ ]:
# !mv json_data/test.* json_data/test

### DATA 변환

##### Mecab 설치

In [15]:
%%capture

# mecab 설치
import os
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.2')
!./configure
!make
!make check
!make install
!ldconfig

# GNU M4 설치
os.chdir('/tmp/')
!wget http://ftp.gnu.org/gnu/m4/m4-1.4.9.tar.gz
!tar -zvxf m4-1.4.9.tar.gz
os.chdir('/tmp/m4-1.4.9')
!./configure
!make
!make install

# autoconf 설치
os.chdir('/tmp')
!curl -OL http://ftpmirror.gnu.org/autoconf/autoconf-2.69.tar.gz
!tar xzf autoconf-2.69.tar.gz
os.chdir('/tmp/autoconf-2.69')
!./configure --prefix=/usr/local
!make
!make install
!export PATH=/usr/local/bin

# automake 설치
os.chdir('/tmp')
!curl -LO http://ftpmirror.gnu.org/automake/automake-1.11.tar.gz
!tar -zxvf automake-1.11.tar.gz
os.chdir('/tmp/automake-1.11')
!./configure
!make
!make install

# mecab dictionary 설치
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
!tar -zxvf mecab-ko-dic-2.1.1-20180720.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.1.1-20180720')
!./autogen.sh
!./configure
!make
!make install

# mecab python 설치
os.chdir('/content')

!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')

!python3 setup.py build
!python3 setup.py install

In [11]:
!pip install kss

  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251547 sha256=1d849724354b9ac15b34d699e836b42553da87ce6d849d4eaba572fa1a17838e
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss


#### dictionary 만들기

In [44]:
tokenizer.parse(sentence)

'담수\tNNG,*,F,담수,*,*,*,*\n와\tJC,*,F,와,*,*,*,*\n염수\tNNG,*,F,염수,*,*,*,*\n가\tJKS,*,F,가,*,*,*,*\n급작\tXR,*,T,급작,*,*,*,*\n스럽\tXSA,*,T,스럽,*,*,*,*\n게\tEC,*,F,게,*,*,*,*\n섞일\tVV+ETM,*,T,섞일,Inflect,VV,ETM,섞이/VV/*+ᆯ/ETM/*\n경우\tNNG,*,F,경우,*,*,*,*\n대부분\tNNG,정적사태,T,대부분,Compound,*,*,대/NNG/*+부분/NNG/*\n의\tJKG,*,F,의,*,*,*,*\n수생\tNNG,*,T,수생,*,*,*,*\n생물\tNNG,*,T,생물,*,*,*,*\n이\tJKS,*,F,이,*,*,*,*\n폐사\tNNG,*,F,폐사,*,*,*,*\n하\tXSV,*,F,하,*,*,*,*\n는\tETM,*,T,는,*,*,*,*\n원인\tNNG,*,T,원인,*,*,*,*\n은\tJX,*,T,은,*,*,*,*\n?\tSF,*,*,*,*,*,*,*\nEOS\n'

In [30]:
from konlpy.tag import Mecab
# mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
tokenizer = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
morph = []
sentence = "한국 감정평가사협회는 진로 교육 체험 기회가 적은 도서지역인 신안 안좌고등학교에서 “나의 미래, 감정평가사?”를 주제로 감동 교실을 열어 감정평가 체험 교육, 부동산 경제 교육, 부동산문화 교육을 진행했다"
for st in sentence.split(" "):
    count = 0
    for token in tokenizer.morphs(st):
        tk = token
        if count > 0:
            tk = "##" + tk
            morph.append(tk)
        else:
            morph.append(tk)
            count += 1
morph        

['한국',
 '감정평가',
 '##사',
 '##협회',
 '##는',
 '진로',
 '교육',
 '체험',
 '기회',
 '##가',
 '적',
 '##은',
 '도서',
 '##지역인',
 '신안',
 '안',
 '##좌고',
 '##등',
 '##학교',
 '##에서',
 '“',
 '##나의',
 '미래',
 '##,',
 '감정평가',
 '##사',
 '##?',
 '##”',
 '##를',
 '주제',
 '##로',
 '감동',
 '교실',
 '##을',
 '열',
 '##어',
 '감정평가',
 '체험',
 '교육',
 '##,',
 '부동산',
 '경제',
 '교육',
 '##,',
 '부동산',
 '##문화',
 '교육',
 '##을',
 '진행',
 '##했',
 '##다']

In [29]:
os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

'D:\\ssum-workspace\\004_mois\\txt_summ_prj\\working\\_git_src\\KoBertSum-master\\ext'

In [31]:
import sys
PROJECT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, PROJECT_DIR)
sys.path.insert(0, PROJECT_DIR + '/src')

In [32]:
SPIECE_UNDERLINE = u'▁'
sentence = SPIECE_UNDERLINE + "담수와 염수가 급작스럽게 섞일 경우 대부분의 수생생물이 폐사하는 원인은?"

In [33]:
sentence

'▁담수와 염수가 급작스럽게 섞일 경우 대부분의 수생생물이 폐사하는 원인은?'

In [35]:
os.path.dirname(os.path.abspath('__file__'))

'D:\\ssum-workspace\\004_mois\\txt_summ_prj\\working\\_git_src\\KoBertSum-master\\ext\\data'

In [48]:
os.chdir(ROOT_DIR)

In [52]:
! cd ,

D:\ssum-workspace\004_mois\txt_summ_prj\working\_git_src\KoBertSum-master\src


In [54]:
from prepro.tokenization_kobert import KoBertTokenizer

tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", do_lower_case=True)
tokenizer.tokenize(sentence)

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert') # monologg/distilkobert도 동일
tokenizer.tokenize(" [CLS] 한국어 모델을 공유합니다. 13243237524972원 [SEP] ")

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

['[CLS]',
 '▁한국',
 '어',
 '▁모델',
 '을',
 '▁공유',
 '합니다',
 '.',
 '▁13',
 '24',
 '32',
 '37',
 '52',
 '49',
 '72',
 '원',
 '[SEP]']

In [55]:
tokenizer.convert_tokens_to_ids(['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]'])

[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [56]:
tokenizer.convert_tokens_to_ids(['[', ']', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]', '13243237524972'])

[362, 363, 6855, 2046, 7088, 1050, 7843, 54, 3, 0]

In [57]:
tokenizer.convert_ids_to_tokens([362, 363, 6855, 2046, 7088, 1050, 7843, 54, 3])

['[', ']', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

#### unused token 찾기

In [59]:
def txt2list(path):
    my_list = []
    file = open(path, "r")
    while True:
        line = file.readline().rstrip("\n")
        if line:
            my_list.append(int(line))
        else:
            break
    return my_list

used_tokens_ids = set(txt2list(r'D:\ssum-workspace\004_mois\txt_summ_prj\working\_git_src\KoBertSum-master\tutorials\used_tokens_ids_kobert.txt'))
unused_tokens_ids = set(range(8002)) - used_tokens_ids
unused_tokens_ids

{1,
 4,
 14,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 48,
 49,
 50,
 51,
 52,
 53,
 57,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 84,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 107,
 120,
 122,
 123,
 124,
 125,
 126,
 130,
 131,
 132,
 143,
 146,
 156,
 158,
 159,
 173,
 174,
 175,
 189,
 203,
 204,
 205,
 206,
 220,
 221,
 235,
 236,
 250,
 251,
 255,
 256,
 260,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345

In [60]:
unused_tokens_ids = set(range(8002)) - used_tokens_ids
tokenizer.convert_ids_to_tokens(unused_tokens_ids)

['[PAD]',
 '[MASK]',
 '&#34;',
 '(0',
 '(1',
 '(10',
 '(12',
 '(15',
 '(17',
 '(18',
 '(19',
 '(2',
 '(20',
 '(23',
 '(24',
 '(25',
 '(3',
 '(4',
 '(5',
 '(6',
 '(7',
 '(8',
 '(9',
 '----------------',
 '-1',
 '-2',
 '-20',
 '-3',
 '-4',
 "...'",
 '0',
 '0%',
 '0%)',
 '0.0',
 '0.00',
 '0.1',
 '0.1%',
 '0.2%',
 '0.3',
 '0.3%',
 '0.4%',
 '0.5',
 '0.5%',
 '0.6',
 '0.6%',
 '0.7',
 '0.7%',
 '0.8',
 '0.8%',
 '000.0',
 '02-',
 '0:00:00',
 '1%',
 '1%)',
 '1)',
 '1,000',
 '1.3%',
 '1.4%',
 '1.5%',
 '1.6',
 '1.6%',
 '1.7%',
 '1.8',
 '1000',
 '2%',
 '2.0',
 '2.3%',
 '2.5',
 '2.5%',
 '2.8%',
 '2011',
 '2012',
 '2013',
 '3%',
 '3000',
 '3⁄4',
 '4%',
 '4%)',
 '5%',
 '5%)',
 '5,000',
 '6%',
 '7%',
 '7%)',
 '7.0',
 '7.5%',
 '8%',
 '8%)',
 '9%',
 '9%)',
 '://',
 ':00',
 '=""',
 '="">',
 '??',
 'A',
 'AM',
 'AP',
 'AR',
 'AS',
 'AT',
 'B',
 'BC',
 'BO',
 'BS',
 'C',
 'CC',
 'CD',
 'CI',
 'D',
 'DB',
 'DC',
 'DI',
 'E',
 'ER',
 'EU',
 'F',
 'FC',
 'FI',
 'FIFA',
 'FTA',
 'G',
 'GB',
 'GDP',
 'GM',
 'H',


In [61]:
tokenizer.convert_tokens_to_ids(['¶','----------------', ';'])

[477, 48, 252]

In [62]:
tokenizer.sp_model.EncodeAsPieces(sentence)

['▁담',
 '수',
 '와',
 '▁염',
 '수가',
 '▁급',
 '작',
 '스럽게',
 '▁',
 '섞',
 '일',
 '▁경우',
 '▁대부분',
 '의',
 '▁수',
 '생',
 '생',
 '물',
 '이',
 '▁폐',
 '사',
 '하는',
 '▁원인',
 '은',
 '?']

In [63]:
text = '[CLS]'
print(tokenizer.sp_model.encode_as_pieces(text))
print(tokenizer.sp_model.encode_as_ids(text))

['▁[', 'C', 'LS', ']']
[249, 425, 4882, 243]


#### make json file

In [64]:
# `[{"src": [["(",],
#            [],
#     ],
#     "tgt": [["(",],
#             [],
#     ]
#   },
#   {}
# ]`

In [65]:
import re
import MeCab
from bs4 import BeautifulSoup
import kss

[Korean Sentence Splitter]: Initializing Kss...


In [66]:
special_symbols_in_dict = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-']
for i in special_symbols_in_dict:
    print(i)

!
"
#
$
%
&
'
(
)
*
+
,
-


In [69]:
# special_symbols_in_dict = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-']
unused_tags = ['SF', 'SE', 'SSO', 'SSC', 'SC', 'SY']

def korean_tokenizer(text, unused_tags=None, print_tag=False): 
    # assert if use_tags is None or unuse_tags is None
    
    tokenizer = MeCab.Tagger(r"-d C:\\mecab\\mecab-ko-dic")
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")]
    result = []
    
    if unused_tags:
        for word_ in word_tag[:-2]:
            word = word_.split("\t")
            tag = word[1].split(",")[0]
            if tag not in unused_tags:
                if print_tag:
                    result.append((word[0], tag))
                else:
                    result.append(word[0]) 
    else:
        for word_ in word_tag[:-2]:
            word = word_.split("\t")
            result.append(word[0]) 

    return result

def number_split(sentence):
    # 1. 공백 이후 숫자로 시작하는 경우만(문자+숫자+문자, 문자+숫자 케이스는 제외), 해당 숫자와 그 뒤 문자를 분리
    num_str_pattern = re.compile(r'(\s\d+)([^\d\s])')
    sentence = re.sub(num_str_pattern, r'\1 \2', sentence)

    # 2. 공백으로 sentence를 분리 후 숫자인경우만 공백 넣어주기
    #numbers_reg = re.compile("\s\d{2,}\s")
    sentence_fixed = ''
    for token in sentence.split():
        if token.isnumeric():
            token = ' '.join(token)
        sentence_fixed+=' '+token
    return sentence_fixed

def noise_remove(text):
    text = text.lower()
    
    # url 대체
    url_pattern = re.compile(r'https?://\S*|www\.\S*')
    text = url_pattern.sub(r'URL', text)

    # html 삭제
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    # 숫자 중간에 공백 삽입하기
    text = number_split(text)
    #number_pattern = re.compile('\w*\d\w*') 
#     number_pattern = re.compile('\d+') 
#     text = number_pattern.sub(r'[[NUMBER]]', text)
    

    # PUCTUACTION_TO_REMOVED = string.punctuation.translate(str.maketrans('', '', '\"\'#$%&\\@'))  # !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ 중 적은것을 제외한 나머지를 삭제
    # text = text.translate(str.maketrans(PUCTUACTION_TO_REMOVED, ' '*len(PUCTUACTION_TO_REMOVED))) 

    # remove_redundant_white_spaces
    text = re.sub(' +', ' ', text)
    
    return text

def preprocessing(text, tokenizer=None, unused_tags=None):
    text = noise_remove(text)
    if tokenizer is not None:
        text = tokenizer(text, unused_tags=unused_tags)
        text = ' '.join(text)

    return text
# korean_tokenizer
text = "■위치 : 당진시 무수동 7길 123 (하늘채코오롱아파트 정문 앞) ■문의 : 041-355-3282! http://dsf/com <a> dsf <a/>"
preprocessing(text, korean_tokenizer, unused_tags=unused_tags)


'위치 당진시 무수동 7 길 1 2 3 하늘 채 코오롱 아파트 정문 앞 문의 0 4 1 355 3282 URL dsf'

In [70]:
for i, row in train_df.iloc[0:100].iterrows():
            print(len(row['article_original']))

26
7
16
11
20
7
11
19
9
12
22
11
8
6
9
14
7
20
13
7
22
15
18
10
9
11
14
21
11
12
7
12
10
12
9
9
11
11
8
13
7
8
13
7
9
9
11
15
7
17
12
17
10
12
9
8
15
9
6
12
11
9
15
9
6
14
17
11
26
7
11
22
9
16
18
13
9
15
7
16
10
14
10
13
9
7
14
7
12
9
16
11
7
21
10
18
9
13
16
8


In [71]:
import json

def create_json_files(df, path='', type=None, summary_target = 'abs'):
    NUM_DOCS_IN_ONE_FILE = 1000
    start_idx_list = list(range(0, len(df), NUM_DOCS_IN_ONE_FILE))

    for start_idx in tqdm(start_idx_list):
        end_idx = start_idx + NUM_DOCS_IN_ONE_FILE
        if end_idx > len(df):
            end_idx = len(df)  # -1로 하니 안됨...

        #정렬을 위해 앞에 0 채워주기
        length = len(str(len(test_df)))
        start_idx_str = (length - len(str(start_idx)))*'0' + str(start_idx)
        end_idx_str = (length - len(str(end_idx-1)))*'0' + str(end_idx-1)

        file_name = os.path.join(path,f"{type}.{start_idx_str}_{end_idx_str}.json")
        
        json_list = []
        for _, row in df.iloc[start_idx:end_idx].iterrows():
            original_sents_list = [preprocessing(original_sent, korean_tokenizer).split()
                                    for original_sent in row['article_original']]

            if summary_target == 'ext':
                summary_sents_list = [preprocessing(original_sent, korean_tokenizer).split()
                                    for original_sent in row['extractive_sents']] if type == 'train' else []
            else:
                summary_sents_list = []
                sents_splited = kss.split_sentences(row['abstractive'])
                if len(sents_splited) == 1:
                    summary_sents_list = sents_splited
                elif len(sents_splited) > 1:
                    is_multi_sent = False
                    for idx in range(len(sents_splited)):
                        if sent_splited[-1] in ['.','?' ]:
                            is_multi_sent = True
                        else:
                            is_multi_sent = False
                            break
                    if is_multi_sent:
                        summary_sents_list.append(sents_splited)
                else:
                    print("error")
                    
            
            json_list.append({'src': original_sents_list,
                              'tgt': summary_sents_list
            })
        # print(json_list)


        json_string = json.dumps(json_list, indent=4, ensure_ascii=False)
        #print(json_string)
        with open(file_name, 'w') as json_file:
            json_file.write(json_string)


In [75]:
summary_sents_list = []

for abstractive in tqdm(train_df['abstractive'].values):
    print(abstractive)
    sents_splited = kss.split_sentences(abstractive)
    
    if len(sents_splited) == 1:
        # .이나 ?가 있는데도 kss가 분리하지 않은 문장들을 혹시나해서 살펴보니
        # 대부분 쉼표나 가운데점 대신 .을 사용하거나 "" 사이 인용문구 안에 들어가있는 점들. -> 괜찮.
        # aa = sents_splited[0].split('. ')
        # if len(aa) > 1:
        #     print(sents_splited)
        summary_sents_list = sents_splited
    elif len(sents_splited) > 1:
        is_multi_sent = False
        summary_sents_list = []
        for idx in range(len(sents_splited) - 1):
            # 두 문장 사이에 .이나 ?가 있는 경우는 그냥 붙여주기
            if sents_splited[idx][-1] in ['.','?' ] and idx != (len(sents_splited) - 1):
                is_multi_sent = True
            else:
                is_multi_sent = False
                break
        if is_multi_sent:
            summary_sents_list.append(sents_splited)

  0%|          | 0/260697 [00:00<?, ?it/s]

통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계를 작성한 이래 가장 많았으며 사망 원인 중 폐렴과 알츠하이머 병의 순위 상승이 두드러졌고 자살에 의한 사망자도 증가했다. 
서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신문협회에서 수여하는 우수의정대상을 수상했는데 가충수 의원은 자동차 연비테스트 연구시설 유치, 천수만 염해피해 재발 방지, 서산시 대표 농산물 육성 등을, 이수의 의원은 지역인재채용 및 관내업체 자재 활용을 확대할 것을 제안하는  다양한 의정활동으로 인정받았다.
 ‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 리더십, 공동체 의식을 함양하는 ‘조선대 국토대장정’이 지난 5일 해단식을 끝으로 마무리했다.
서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으로 무상급식을 실시하고 이를 2학년, 1학년 순으로 확대 실시한다고 밝혔다.
미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 시민단체와 주민들이 고소한 가운데, CNN은 3일(현지시간) 우간다에서 비영리 종교단체를 설립한 러네이 바흐가 무허가로 의료 시설을 운영한 혐의로 우간다 여성 인권 단체로부터 고소당했다고 전했다.
음성 극동대학교 평생교육권은 12일 극동대 혁신도시센터에서 어린 자녀가 있는 부모를 대상으로 '내 아이를 위한 만화심리치료' 강좌를 개설하였으며 강좌를 진행하는 김동욱 원장은 그동안 터득한 노하우를 지역주민들에게 전파하고 있다.
19일 아침 충남 당진우체국 집배원이 또 사망, 올해에만 9번째 과로사가 발생한 가운데 우정 노조는 이에 정부가 집배원인력증원과 완전한 주5일제를 보장하고 이행을 위해 직접 나서라고 촉구했다
한국철도기술연구원의 '무가선 저상트램 실증노선 공모'에서 제1호 트램도시로 선정되기 위해 수원시와 성남시가 치열한 경쟁 구도를 보이고 있다.
위니아딤채는 '우리집 딤채를 소개합니다'라는 컨셉으로 제작한 

트로피컬 요소가 가미되어 트랩 비트가 인상적인 화사의 첫 솔로곡 '멍청이'가 지난 13일 공개되어 주요 음원 사이트 1위에 올랐으며, 캐나다, 프랑스 등 전 세계 11개국 아이튠즈 K팝 싱글 차트 1위를 기록하며 국내외 차트에서 돌풍을 일으키고 있다.
연극 '최고의 사랑은' 여섯살 예솔이의 엄마에 대한 그리움,가족에 대한 애틋함과 15년 절친인 정복에 대한 사랑 등 다양한 이야기로 웃음과 감동을 선사하였으며 이날 가족과 관람한 한 시민은 사랑에 관한 에피소드를 통해 사랑에 대한 최고의 순간에 대해 생각해 보는 시간이 되었다고 평가하였다.
더불어민주당은 최대 25%, 자유한국당은 최대 50% 등 정치 신인에게 공천시 가산점을 주는 공천룰을 확정하였지만 낮은 인지도를 극복하기 어렵고 꾸준히 준비해온 현역 의원을 상대하기가 어렵다는 한계가 있다.
정부의 소규모화 정책에 따라 시설을 분리 운영하고 있는 지적장애인아동시설이 시설 분리 1년 후부터 정부 보조금을 지급받지 못해 문을 닫을 위기에 처해 있다.
아이를 경기지역 사립유치원에 보내는 학부모의 교육비 부담액은 월 23만 2,907원으로 전국 평균 부담비(19만 5,119원)보다 3만 7,788원 더 많았으며 지역별로는 성남시 분당이 약 26만원으로 가장 높았고 다음은 용인시 수지가 약 24만원, 김포시가 약 23만원대로 뒤를 이었다.
서울고등법원 민사9부는 응급처치 중 사망한 유족의 손해배상 소송에 일부 승소 판결을 내려 위자료 9500만원을 지급하라고 지시한 1심에 문제가 없다고 판단하고 유족과 병원의 항소를 기각했다.
매년 전 직원이 동참해 물품들을 기증하고 수익금을 지역사회에 환원하는 행사를 진행하는 (재)군포문화재단은 올해에도 아름다운가게 군포점과 지난 25일 나눔행사를 개최했으며, 발생한 수익금은 군포의왕교육지원청을 통해 초·중·고교생에게 장학금으로 기부한다.
서산시가 3일 베니키아호텔서산에서 맹정호 시장 주재로 500여명의 시민이 참석한 가운데 최대 현안으로 떠오른 '터미널 이전·수석지구도시개발사업에 관

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
create_json_files(train_df, path=JSON_DATA_DIR + '/train_abs', type='train')

KeyboardInterrupt: ignored

In [ ]:
create_json_files(test_df, path=JSON_DATA_DIR, type='test')

#### 4. Make .json file as .pt file for BERT model 

In [ ]:
!pip install -r $PreSumm_DIR/requirements.txt
!pip install transformers

In [ ]:
import os
os.chdir(PreSumm_DIR + '/src/')

# 동일한 파일명 존재하면 덮어쓰는게 아니라 넘어감
!python preprocess.py -mode format_to_bert \
                        -raw_path $JSON_DATA_DIR/train_abs -save_path $BERT_DATA_DIR/train_abs  -lower -n_cpus 1 -log_file $LOG_FILE \
                        -dataset train

Traceback (most recent call last):
  File "preprocess.py", line 8, in <module>
    from prepro import data_builder
  File "/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/src/prepro/data_builder.py", line 18, in <module>
    from others.tokenization_kobert import KoBertTokenizer
  File "/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/src/others/tokenization_kobert.py", line 23, in <module>
    from transformers import PreTrainedTokenizer
ModuleNotFoundError: No module named 'transformers'


In [ ]:
import os
os.chdir(PreSumm_DIR + '/src/')

# 동일한 파일명 존재하면 덮어쓰는게 아니라 넘어감
!python preprocess.py -mode format_to_bert \
                        -raw_path $JSON_DATA_DIR -save_path $BERT_DATA_DIR  -lower -n_cpus 1 -log_file $LOG_FILE \
                        -dataset test

2020-11-30 05:25:04.736700: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
[('test', '/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/json_data/test.0000_0099.json', Namespace(dataset='test', log_file='/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/logs/ext_bert.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/json_data', save_path='/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/bert_data', select_mode='greedy', shard_size=2000, use_bert_basic_tokenizer=False), '/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/be

#### (이전) sum -> .story data -> tokenized -> .json file


In [ ]:
# original_sents_list_sr = train_df['article_original']
# abstractive_sents_sr = train_df['abstractive']
# extractive_sents_list_sr = train_df['extractive_sents']

`temp_ext.raw_src`
- insert [CLS] [SEP] as your sentence boundaries.
- example   
this Terry Jones had a love of the absurd that contributed much to the anarchic humour of Monty Python's Flying Circus. [CLS] [SEP] His style of visual comedy, leavened with a touch of the surreal, inspired many comedians who followed him. 
(CNN) An Iranian chess referee says she is frightened to return home after she was criticized online for not wearing the appropriate headscarf during an international tournament. [CLS] [SEP] CNN contacted the Iranian Chess Federation on Tuesday but has yet to receive a response.


`temp.raw_tgt` 
- 순서에 맞춰서 한 줄씩

In [ ]:
# original_docs = ''

# for original_sents_list in original_sents_list_sr:
#     original_doc = ''
#     for idx, original_sent in enumerate(original_sents_list):
#         original_sent = str.strip(original_sent)
#         if idx == len(original_sents_list)-1:
#             original_doc += original_sent + '\n'
#         else:
#             original_doc += original_sent + ' [CLS] [SEP] ' 
#     #print(original_doc)
#     original_docs += original_doc
# #original_docs


# with open(PreSumm_DIR + '/raw_data/temp.raw_tgt', 'w') as file:
#     file.write(original_docs)

In [ ]:
# abstractive_docs = ''
# for abstractive_sents in abstractive_sents_sr:
#     abstractive_docs += abstractive_sents + '\n'

# with open(PreSumm_DIR + '/raw_data/temp_ext.raw_src', 'w') as file:
#     file.write(abstractive_docs)

In [ ]:
def joiner(sents_list, sent_type='origin'):
    if sent_type == 'origin':
        return '\n\n'.join(sents_list)
    else:  # summary
        summary = ''
        for sent in sents_list:
            summary += '\n\n@highlight\n\n' + sent
        return summary
joiner(train_df['extractive_sents'][1], 'summary')

NameError: ignored

In [ ]:

def create_story_files(df,num_file=None,filename=None):
    if num_file == None:
        num_file = len(df)
    for i in tqdm(range(num_file)):
        doc = joiner(train_df['article_original'][i]) \
            +  joiner(train_df['extractive_sents'][i], 'summary')

        file_name = os.path.join(filename,(str(i) + '.story'))
        with open(file_name,'w') as story_file:
            story_file.write(doc)

In [ ]:
# create_story_files(train_df,filename=STORY_DIR)

In [ ]:
# Download Stanford CoreNLP
# We will need Stanford CoreNLP to tokenize the data. Download it here and unzip it. Then add the following command to your bash_profile:

!export CLASSPATH=$PreSumm_DATA_DIR/stanford-corenlp-4.2.0/stanford-corenlp-4.2.0.jar

In [ ]:
import os
os.chdir(PreSumm_DIR + '/src/')

!python preprocess.py -mode tokenize -raw_path $STORY_DIR -save_path $MERGE_STORY_DIR  -log_file $LOG_FILE

Preparing to tokenize /content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/raw_stories to /content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/merged_stories_tokenized...
Making list of files to tokenize...
Tokenizing 42803 files in /content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/raw_stories and saving in /content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/data/merged_stories_tokenized...
Error: Could not find or load main class edu.stanford.nlp.pipeline.StanfordCoreNLP
Caused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP
Stanford CoreNLP Tokenizer has finished.
Traceback (most recent call last):
  File "preprocess.py", line 73, in <module>
    eval('data_builder.'+args.mode + '(args)')
  File "<string>", line 1, in <module>
  File "/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/src/prepro/data_builder.py", 

In [ ]:
# 3. Make tokenized data files as .json file
!python preprocess.py -mode format_to_lines -raw_path $STORY_DIR -save_path $JSON_DATA_DIR -n_cpus 1 -use_bert_basic_tokenizer false -log_file $LOG_FILE


Traceback (most recent call last):
  File "preprocess.py", line 73, in <module>
    eval('data_builder.'+args.mode + '(args)')
  File "<string>", line 1, in <module>
  File "/content/drive/MyDrive/Notebooks/Projects/202011_DACON_Summarization/PreSumm/src/prepro/data_builder.py", line 335, in format_to_lines
    for line in open(pjoin(args.map_path, 'mapping_' + corpus_type + '.txt')):
FileNotFoundError: [Errno 2] No such file or directory: '../../data/mapping_valid.txt'


## Rouge

In [115]:
from others.test_rouge_score import RougeScorer
from others.rouge_metric import Rouge

In [113]:
rouge_scorer = RougeScorer()
#rouge_scorer.compute_rouge(self, ref_df, hyp_df):

In [219]:
rouge_evaluator = Rouge(
            metrics=["rouge-n", "rouge-l"],
            max_n=2,
            limit_length=True,
            length_limit=1000,
            length_limit_type="words",
            use_tokenizer=True,
            apply_avg=True,
            apply_best=False,
            alpha=0.5,  # Default F1_score
            weight_factor=1.2,
        )

# 문장 순서에 따라 점수 달라짐
generated_summaries = "어디로 잘 갔어요? 오늘은 ! ! ! 집에 ㅎㅎ 갔다. "
reference_summaries = "집에 잘 들어갔어?"
rouge_evaluator.get_scores(generated_summaries, reference_summaries)

{'rouge-2': {'f': 0.14285714285714288, 'p': 0.1, 'r': 0.25},
 'rouge-1': {'f': 0.37499999999999994, 'p': 0.2727272727272727, 'r': 0.6},
 'rouge-l': {'f': 0.25000000000000006, 'p': 0.18181818181818182, 'r': 0.4}}

In [120]:
rouge_scores = rouge_evaluator.get_scores(generated_summaries, reference_summaries)
total_rouge_score = 0
for k, v in rouge_scores.items():
    total_rouge_score += v['f']
total_rouge_score

0.9494949494949494

In [129]:
from prepro.utils import _get_word_ngrams

In [157]:
sents = [['노후', '석탄화력발전소의', '수명', '연장을', '추진했던', '한국동서발전이', '당진화력', '14호기의', '수명연장을', '하지', '않기로', '결정했다'], ['지난달', '열린', '이사회에서', '한국동서발전은', '당진화력', '14호기의', '수명연장을', '하지', '않고', '여수에', '위치한', '호남화력본부와', '울산화력본부에', '각각', '1000mw급의', 'lng발전소를', '건설하는', '내용의', '제9차', '전력수급기본계획', '관련', '대체', '건설', '의향서', '제출안을', '의결한', '것으로', '알려졌다'], ['한편', '당진화력', '14호기는', '지난', '1999년', '6월부터', '2001년', '3월', '사이에', '차례대로', '준공했다'], ['14호기의', '설계수명이', '다가오는', '2029년부터', '2031년까지', '순차적으로', '폐쇄해야', '하는', '상황인', '것이다'], ['이에', '한국동서발전은', '지난해', '말', '당진화력', '14호기의', '성능개선', '사업의', '경제적', '타당성을', '평가하면서', '기존', '30년이었던', '설계수명을', '40년으로', '가정해', '논란이', '일었다'], ['환경단체에서는', '이', '를', '추진한', '한국동서발전에', '대해', '노후', '석탄화력발전소의', '성능개선', '사업이', '아닌', '수명연장을', '추진하는', '꼼수라고', '비판했다'], ['당시', '환경단체는', '물론', '양승조', '충남도지사와', '김홍장', '당진시장', '또한', '대기오염물질', '배출을', '줄이기', '위한', '노후', '화력발전소의', '환경설비', '강화는', '전적으로', '찬성하지만', '발전기와', '보일러', '등', '주설비를', '개조해', '설계수명을', '10년', '더', '연장하는', '것에는', '반대한다며', '강경한', '입장을', '밝힌', '바', '있다']]
sents_merge = [' '.join(sent) for sent in sents]
sents_merge

['노후 석탄화력발전소의 수명 연장을 추진했던 한국동서발전이 당진화력 14호기의 수명연장을 하지 않기로 결정했다',
 '지난달 열린 이사회에서 한국동서발전은 당진화력 14호기의 수명연장을 하지 않고 여수에 위치한 호남화력본부와 울산화력본부에 각각 1000mw급의 lng발전소를 건설하는 내용의 제9차 전력수급기본계획 관련 대체 건설 의향서 제출안을 의결한 것으로 알려졌다',
 '한편 당진화력 14호기는 지난 1999년 6월부터 2001년 3월 사이에 차례대로 준공했다',
 '14호기의 설계수명이 다가오는 2029년부터 2031년까지 순차적으로 폐쇄해야 하는 상황인 것이다',
 '이에 한국동서발전은 지난해 말 당진화력 14호기의 성능개선 사업의 경제적 타당성을 평가하면서 기존 30년이었던 설계수명을 40년으로 가정해 논란이 일었다',
 '환경단체에서는 이 를 추진한 한국동서발전에 대해 노후 석탄화력발전소의 성능개선 사업이 아닌 수명연장을 추진하는 꼼수라고 비판했다',
 '당시 환경단체는 물론 양승조 충남도지사와 김홍장 당진시장 또한 대기오염물질 배출을 줄이기 위한 노후 화력발전소의 환경설비 강화는 전적으로 찬성하지만 발전기와 보일러 등 주설비를 개조해 설계수명을 10년 더 연장하는 것에는 반대한다며 강경한 입장을 밝힌 바 있다']

In [152]:
def permute(a, l, r): 
    if l==r: 
        print(a)
    else: 
        for i in range(l,r+1): 
            a[l], a[i] = a[i], a[l] 
            permute(a, l+1, r) 
            a[l], a[i] = a[i], a[l] # backtrack 
 
# Driver program to test the above function 
sents_merge = [1,2,3,4,5]
n = len(sents_merge) 
permute(sents_merge, 0, n-1) 

[1, 2, 3, 4, 5]
[1, 2, 3, 5, 4]
[1, 2, 4, 3, 5]
[1, 2, 4, 5, 3]
[1, 2, 5, 4, 3]
[1, 2, 5, 3, 4]
[1, 3, 2, 4, 5]
[1, 3, 2, 5, 4]
[1, 3, 4, 2, 5]
[1, 3, 4, 5, 2]
[1, 3, 5, 4, 2]
[1, 3, 5, 2, 4]
[1, 4, 3, 2, 5]
[1, 4, 3, 5, 2]
[1, 4, 2, 3, 5]
[1, 4, 2, 5, 3]
[1, 4, 5, 2, 3]
[1, 4, 5, 3, 2]
[1, 5, 3, 4, 2]
[1, 5, 3, 2, 4]
[1, 5, 4, 3, 2]
[1, 5, 4, 2, 3]
[1, 5, 2, 4, 3]
[1, 5, 2, 3, 4]
[2, 1, 3, 4, 5]
[2, 1, 3, 5, 4]
[2, 1, 4, 3, 5]
[2, 1, 4, 5, 3]
[2, 1, 5, 4, 3]
[2, 1, 5, 3, 4]
[2, 3, 1, 4, 5]
[2, 3, 1, 5, 4]
[2, 3, 4, 1, 5]
[2, 3, 4, 5, 1]
[2, 3, 5, 4, 1]
[2, 3, 5, 1, 4]
[2, 4, 3, 1, 5]
[2, 4, 3, 5, 1]
[2, 4, 1, 3, 5]
[2, 4, 1, 5, 3]
[2, 4, 5, 1, 3]
[2, 4, 5, 3, 1]
[2, 5, 3, 4, 1]
[2, 5, 3, 1, 4]
[2, 5, 4, 3, 1]
[2, 5, 4, 1, 3]
[2, 5, 1, 4, 3]
[2, 5, 1, 3, 4]
[3, 2, 1, 4, 5]
[3, 2, 1, 5, 4]
[3, 2, 4, 1, 5]
[3, 2, 4, 5, 1]
[3, 2, 5, 4, 1]
[3, 2, 5, 1, 4]
[3, 1, 2, 4, 5]
[3, 1, 2, 5, 4]
[3, 1, 4, 2, 5]
[3, 1, 4, 5, 2]
[3, 1, 5, 4, 2]
[3, 1, 5, 2, 4]
[3, 4, 1, 2, 5]
[3, 4, 1, 5, 2]
[3, 4, 2

In [160]:
from itertools import permutations 
l = list(permutations(range(len(sents_merge)), 3)) 
print(l)

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 2, 1), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 3, 1), (0, 3, 2), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 4, 5), (0, 4, 6), (0, 5, 1), (0, 5, 2), (0, 5, 3), (0, 5, 4), (0, 5, 6), (0, 6, 1), (0, 6, 2), (0, 6, 3), (0, 6, 4), (0, 6, 5), (1, 0, 2), (1, 0, 3), (1, 0, 4), (1, 0, 5), (1, 0, 6), (1, 2, 0), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 2, 6), (1, 3, 0), (1, 3, 2), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 4, 0), (1, 4, 2), (1, 4, 3), (1, 4, 5), (1, 4, 6), (1, 5, 0), (1, 5, 2), (1, 5, 3), (1, 5, 4), (1, 5, 6), (1, 6, 0), (1, 6, 2), (1, 6, 3), (1, 6, 4), (1, 6, 5), (2, 0, 1), (2, 0, 3), (2, 0, 4), (2, 0, 5), (2, 0, 6), (2, 1, 0), (2, 1, 3), (2, 1, 4), (2, 1, 5), (2, 1, 6), (2, 3, 0), (2, 3, 1), (2, 3, 4), (2, 3, 5), (2, 3, 6), (2, 4, 0), (2, 4, 1), (2, 4, 3), (2, 4, 5), (2, 4, 6), (2, 5, 0), (2, 5, 1), (2, 5, 3), (2, 5, 4), (2, 5, 6), (2, 6, 0), (2, 6, 1), (2, 6, 3), (2, 6, 4), (2, 6, 5), (3, 0, 1)

In [215]:
def full_selection(doc_sent_list, abstract_sent_list, summary_size=3):
    def _rouge_clean(s):
        return re.sub(r'[^A-Za-z0-9가-힣 ]', '', s)

    rouge_evaluator = Rouge(
            metrics=["rouge-n", "rouge-l"],
            max_n=2,
            limit_length=True,
            length_limit=1000,
            length_limit_type="words",
            use_tokenizer=True,
            apply_avg=True,
            apply_best=False,
            alpha=0.5,  # Default F1_score
            weight_factor=1.2,
        )

    # cleaning and merge [[w,w,w], [w,w,w]] -> [w,w,w, w,w,w] 
    abstract = sum(abstract_sent_list, [])  # [[w,w,w], [w,w,w]] -> [w,w,w, w,w,w] 
    abstract = _rouge_clean(' '.join(abstract))
    doc_sent_list_merged = [_rouge_clean(' '.join(sent)) for sent in doc_sent_list]
    src_len = len(doc_sent_list_merged)

    # 일단 greedy로 구한 다음 3개가 안되는 경우만 나머지를 full로 채움!
    selected_idx3_list = greedy_selection(doc_sent_list, abstract_sent_list, summary_size)
    print(selected_idx3_list)
    if len(selected_idx3_list) == 3:
        return selected_idx3_list

    total_max_rouge_score = 0.0
    if src_len > 10 or (src_len < 10 and len(selected_idx3_list) < 2): # greedy
        for i in range(summary_size - len(selected_idx3_list)):

            #cur_sents_idx3_list = []
            cur_max_total_rouge_score = 0.0
            cur_sent_idx = -1
            for sent_idx, sent in enumerate(doc_sent_list_merged):
                
                if sent_idx in selected_idx3_list:
                    continue
                temp_idx3_list = selected_idx3_list + [sent_idx]
                sents_array = np.array(doc_sent_list_merged)[temp_idx3_list]
                sents_merged = ' '.join(sents_array)
                #print('temp_idx3_list', temp_idx3_list)
                # ROUGE1,2,l 합score 계산
                rouge_scores = rouge_evaluator.get_scores(sents_merged, abstract)
                total_rouge_score = 0
                for k, v in rouge_scores.items():
                    total_rouge_score += v['f']
                # print('total_rouge_score', total_rouge_score)
                if total_rouge_score > cur_max_total_rouge_score:
                    cur_max_total_rouge_score = total_rouge_score
                    cur_sent_idx = sent_idx
                  #  print(cur_max_total_rouge_score)
                   # print(selected_idx3_list)
            selected_idx3_list.append(cur_sent_idx)
            # print('-----------------------')
        total_max_rouge_score = cur_max_total_rouge_score
    print(selected_idx3_list)      
            
    # full
    sents_idx_perm_list = list(permutations(range(src_len), summary_size)) 
    sents_idx_list = []
    for sents_idx_perm in sents_idx_perm_list:
        if set(sents_idx_perm) & set(selected_idx3_list) == set(selected_idx3_list):
            sents_idx_list.append(sents_idx_perm)

    for sents_idx in sents_idx_list:
        sents_array = np.array(doc_sent_list_merged)[list(sents_idx)]
        sents_merged = ' '.join(sents_array)
        #print(sents_merged)
        # print(sents_idx)
        # print(sents_array)

        # ROUGE1,2,l 합score 계산
        rouge_scores = rouge_evaluator.get_scores(sents_merged, abstract)
        total_rouge_score = 0
        for k, v in rouge_scores.items():
            total_rouge_score += v['f']

        if total_rouge_score > total_max_rouge_score:
            total_max_rouge_score = total_rouge_score
            selected_idx3_list = list(sents_idx)

    return selected_idx3_list #, total_max_rouge_score,  sorted(selected_idx3_list)

full_selection(sents, [['노후', '석탄화력발전소의', '수명', '연장을', '추진했던']], 3)

[4, 2]
[4, 2]


[0, 2, 4]

In [221]:
a=  ["[청주]청주시의회 복지교육위원회(위원장 김은숙)는 16일 산남종합사회복지관 주관으로 개최된 '1인가구 중장년 고독사 예방을 위한 지역사회 유관기관 및 관계자 간담회'에서 중·장년층 고독사 예방을 위해 조례 개정이 필요하다고 밝혔다.", "현재 청주시에는 홀로 사는 노인의 고독사를 예방하기 위한 '청주시 홀로 사는 노인 고독사 예방을 위한 조례'가 지난 2017년 제정, 시행되고 있다.", '하지만 최근 장년층의 고독사가 증가함에 따라 장년층 고독사 예방을 위한 조례 개정이 필요하다는 지적이 지속적으로 제기돼 왔다.', '이와 관련, 김은숙 복지교육위원장은 "최근 실업, 빈곤, 이혼 등으로 인한 가족구조의 변화로 노인 및 장년층의 1인 가구가 급속히 증가하고 있다"면서 "60대 이상 1인 중장년 가구는 상대적으로 소외된 삶을 살며 이들이 가족과 지역사회에서 제대로 된 도움을 받지 못하고 외롭게 죽음을 맞이하는 사례가 증가하는 등 노인 및 장년층의 고독사가 심각한 사회문제로 대두되고 있다"며 중·장년층 고독사 예방을 위한 조례 개정의 필요성을 강조했다.', '이재숙 의원도 "홀로 사는 노인들에 대한 고독사 뿐만 아니라 60대 이상 장년층에 대한 고독사 문제도 깊이 고민해봐야 한다"며 조례 개정의 필요성에 힘을 보탰다.', '시의회 관계자는 "이날 간담회에 참석한 복지교육위원회 위원들은 조례 개정의 필요성에 깊이 공감하고 다양한 의견을 적극적으로 수렴해 조례를 개정할 뜻을 밝혔다"고 말했다.']
b=  [0, 2, 1, 4, 3, 5]

np.array(a)[b[:3]]

array(["[청주]청주시의회 복지교육위원회(위원장 김은숙)는 16일 산남종합사회복지관 주관으로 개최된 '1인가구 중장년 고독사 예방을 위한 지역사회 유관기관 및 관계자 간담회'에서 중·장년층 고독사 예방을 위해 조례 개정이 필요하다고 밝혔다.",
       '하지만 최근 장년층의 고독사가 증가함에 따라 장년층 고독사 예방을 위한 조례 개정이 필요하다는 지적이 지속적으로 제기돼 왔다.',
       "현재 청주시에는 홀로 사는 노인의 고독사를 예방하기 위한 '청주시 홀로 사는 노인 고독사 예방을 위한 조례'가 지난 2017년 제정, 시행되고 있다."],
      dtype='<U244')

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForNextSentencePrediction.from_pretrained('monologg/kobert', return_dict=True)

0 indicates sequence B is a continuation of sequence A,

1 indicates sequence B is a random sequence.

In [234]:
prompt = "오늘은 소풍날이다."
next_sentence = "여러 친구들과 함께 소풍을 갔다."
encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

outputs = model(**encoding, labels=torch.LongTensor([1]))
print(outputs)

logits = outputs.logits
print(logits[0, 0])
print(logits[0, 1]) # next sentence was random
# >>> assert logits[0, 0] < logits[0, 1] # next sentence was random

NextSentencePredictorOutput(loss=tensor(0.7691, grad_fn=<NllLossBackward>), logits=tensor([[-0.0543, -0.2008]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
tensor(-0.0543, grad_fn=<SelectBackward>)
tensor(-0.2008, grad_fn=<SelectBackward>)


In [266]:
sent_list =["[청주]청주시의회 복지교육위원회(위원장 김은숙)는 16일 산남종합사회복지관 주관으로 개최된 '1인가구 중장년 고독사 예방을 위한 지역사회 유관기관 및 관계자 간담회'에서 중·장년층 고독사 예방을 위해 조례 개정이 필요하다고 밝혔다.",
       '하지만 최근 장년층의 고독사가 증가함에 따라 장년층 고독사 예방을 위한 조례 개정이 필요하다는 지적이 지속적으로 제기돼 왔다.',
       "현재 청주시에는 홀로 사는 노인의 고독사를 예방하기 위한 '청주시 홀로 사는 노인 고독사 예방을 위한 조례'가 지난 2017년 제정, 시행되고 있다."]
m = torch.nn.Softmax(dim=1)

perm_idxes= list(permutations(range(3), 2)) 
perm_idxes_score = []
for idxes in perm_idxes:
    print(idxes)
    sub_sent_list = np.array(sent_list)[list(idxes)]

    encoding = tokenizer(sub_sent_list[0], sub_sent_list[1], return_tensors='pt')

    outputs = model(**encoding)  # , labels=torch.LongTensor([1])
    #print(outputs)

    logits = outputs.logits
    score = m(logits)[0,0]
    #print(logits)
    print(score)
    
#     score = logits[0, 0] / (logits[0, 0]+logits[0, 1])
#     print(score)
    # >>> assert logits[0, 0] < logits[0, 1] # next sentence was random

(0, 1)
tensor(0.5388, grad_fn=<SelectBackward>)
(0, 2)
tensor(0.5432, grad_fn=<SelectBackward>)
(1, 0)
tensor(0.5461, grad_fn=<SelectBackward>)
(1, 2)
tensor(0.5410, grad_fn=<SelectBackward>)
(2, 0)
tensor(0.5415, grad_fn=<SelectBackward>)
(2, 1)
tensor(0.5370, grad_fn=<SelectBackward>)


In [236]:
from itertools import permutations 
l = list(permutations(range(3), 2)) 
print(l)

[(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]


In [244]:
sent_list =["[청주]청주시의회 복지교육위원회(위원장 김은숙)는 16일 산남종합사회복지관 주관으로 개최된 '1인가구 중장년 고독사 예방을 위한 지역사회 유관기관 및 관계자 간담회'에서 중·장년층 고독사 예방을 위해 조례 개정이 필요하다고 밝혔다.",
       '하지만 최근 장년층의 고독사가 증가함에 따라 장년층 고독사 예방을 위한 조례 개정이 필요하다는 지적이 지속적으로 제기돼 왔다.',
       "현재 청주시에는 홀로 사는 노인의 고독사를 예방하기 위한 '청주시 홀로 사는 노인 고독사 예방을 위한 조례'가 지난 2017년 제정, 시행되고 있다."]
np.array(sent_list)[list((1,2))]

array(['하지만 최근 장년층의 고독사가 증가함에 따라 장년층 고독사 예방을 위한 조례 개정이 필요하다는 지적이 지속적으로 제기돼 왔다.',
       "현재 청주시에는 홀로 사는 노인의 고독사를 예방하기 위한 '청주시 홀로 사는 노인 고독사 예방을 위한 조례'가 지난 2017년 제정, 시행되고 있다."],
      dtype='<U129')